MARKET POTENTIAL

In [1]:
from IPython.display import HTML

HTML("""
<style>
.jp-Cell-outputArea, .output_area {
    overflow: visible !important;
}
</style>
""")


In [2]:
import endpoints
import random
geoserver = endpoints.Geoserver(
        url="https://geoserver-4vzmi.ondigitalocean.app/geoserver",
        collections_endpoint="ogc/features/v1/collections")
adresses = "kortreist:addresser_leilighet"
tettsteder_aggr = "kortreist:tettsteder2025_aggr"
tettsteder = "kortreist:tettsteder2025"
tettsteder_centroids = "kortreist:tettsteder2025_centroids"
tettsteder_list=endpoints.get_unique_values_from_geoserver_collection(geoserver, tettsteder_centroids, "tettstednavn")
random_item = random.choice(tettsteder_list)
print(random_item)
#random_item = "Sarpsborg"
tettsted_centroid=endpoints.get_features_by_attribute_value(geoserver, tettsteder_centroids, "tettstednavn", random_item)
tettsted_polygon_aggr=endpoints.get_features_by_attribute_value(geoserver, tettsteder_aggr, "tettstednavn", random_item)
print(tettsted_centroid['geometry'])
print(tettsted_polygon_aggr['geometry'][0].geoms)


Haugland
0    POINT (4.93203 60.69933)
Name: geometry, dtype: geometry


In [3]:
import openrouteservice
from ipyleaflet import Map, basemaps, Marker,AwesomeIcon, Popup,GeoJSON
from ipywidgets import HTML
from shapely.geometry import mapping
from ipywidgets import Layout

icon = AwesomeIcon(
    name='flag',
    marker_color='green',
    icon_color='white'
)

center = [tettsted_centroid.geometry.y.iloc[0], tettsted_centroid.geometry.x.iloc[0]]
zoom = 14
label=HTML(random_item)
popup = Popup(location=center, child=label, close_button=False, auto_close=False, close_on_escape_key=False)
marker = Marker(location=center, draggable=True,icon=icon,popup=popup)
layout = Layout(width="100%", height="500px")
m=Map(basemap=basemaps.OpenStreetMap.Mapnik, center=center, zoom=zoom,layout = layout)
m.add(marker)

tettsted_polygon = mapping(tettsted_polygon_aggr['geometry'][0])
layer = GeoJSON(
    data=tettsted_polygon,
    style={
        "color": "blue",
        "weight": 2,
        "fillColor": "lightblue",
        "fillOpacity": 0.5
    },
    hover_style={"fillColor": "yellow", "fillOpacity": 0.7},
)
m.add(layer)
display(m)


Map(center=[np.float64(60.69932532), np.float64(4.93202632)], controls=(ZoomControl(options=['position', 'zoom…

In [4]:
print(marker.location)
coordinate=[marker.location[1],marker.location[0]]


[np.float64(60.69932532), np.float64(4.93202632)]


In [5]:
import features
import endpoints
from ipyleaflet import GeoJSON

iso=features.get_isochrone_time(marker.location[1],marker.location[0],10)
addresses=features.get_addresses_from_osm_for_isochrone(iso)
addresses=endpoints.get_features_by_attribute_value(geoserver, adresses, "tettstednavn", random_item)
isochrone_layer = GeoJSON(data=iso, style={"color": "green"})
subsampled=features.subsample_addresses(addresses,25)
popfraction=len(subsampled)/len(addresses)

addres_layer =GeoJSON(data=features.gdf_to_geojson(subsampled),point_style={
        "radius": 8,           # circle size
        "color": "blue",       # outline color
        "fillColor": "yellow", # fill color
        "fillOpacity": 0.8,    # transparency
        "weight": 2            # outline width
    },)
route,stops,summary=features.get_route(subsampled)
route_layer =GeoJSON(data=route,style={"color": "red"})
dispatches=features.coordinates_to_geojson_with_labels(stops)
m.add(GeoJSON(data=dispatches))
m.add(isochrone_layer)
m.add(addres_layer)
m.add(route_layer)


KeyboardInterrupt: 

In [ ]:
from datetime import timedelta
print(marker.location)
coordinate=[marker.location[1],marker.location[0]]
total_seconds = summary['durationprint(marker.location)
coordinate=[marker.location[1],marker.location[0]]']
dispatch_time = len(stops)*120
time_str = str(timedelta(seconds=total_seconds))
timetotal= str(timedelta(seconds=dispatch_time+total_seconds))
distance = summary['distance']/1000
print(len(stops))
print(f"Traveled distance: {distance} km, traveled time: {time_str}, total time: {timetotal}")
print(f"fraction of population in the area: {popfraction*100}% from {len(addresses)} addresses")